In [11]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [12]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

Conectando ao Neo4j 

In [13]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [21]:
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository[1000:1100].reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:a_percepcao_da_injuria_racial_...,http://www.teses.usp.br/teses/disponiveis/2/21...
1,tag:stardog:api:nacionalismos_antiracistas_man...,https://www.teses.usp.br/teses/disponiveis/8/8...
2,tag:stardog:api:valores_e_sentimentos_subjacen...,http://www.teses.usp.br/teses/disponiveis/48/4...
3,tag:stardog:api:politicas_e_praticas_de_saude_...,http://www.teses.usp.br/teses/disponiveis/108/...
4,tag:stardog:api:os_limites_dos_direitos_humano...,http://www.teses.usp.br/teses/disponiveis/2/21...
...,...,...
95,tag:stardog:api:estimativa_de_parametros_genet...,http://www.teses.usp.br/teses/disponiveis/41/4...
96,tag:stardog:api:agroecologia_quilombola_ou_qui...,http://www.teses.usp.br/teses/disponiveis/8/81...
97,tag:stardog:api:conflitos_fundiarios_e_meio_am...,http://www.teses.usp.br/teses/disponiveis/8/81...
98,tag:stardog:api:entre_o_rio_e_o_asfalto_as_dis...,https://www.teses.usp.br/teses/disponiveis/8/8...


Criando as utils functions

In [22]:

# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese 1.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  
      
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        
        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [23]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link}")
        download_file(pdf_link, thesis_uri)
        
        # Extraindo o texto e entidades
        tese_path = 'tese 1.pdf'
        excerpt = extratc_text(tese_path)

        excerpts = []
        persons = []
        gpes = []

        # Salvando as triplas em CSV
        for n in excerpt.keys():

            excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
            excerpt_text = excerpt[n]['text']
            if "'" in excerpt_text:
                excerpt_text = excerpt_text.replace("'", "")

            excerpt_page = n
            excerpt_lang = excerpt[n]['lang']
            
            excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, str(excerpt_page), excerpt_lang])
                
            #Criando as relações com as entidades
            for person in excerpt[n]['persons']:
                persons.append([excerpt_uri, 'tag:stardog:api:' + person])
            
            for gpe in excerpt[n]['gpes']:
                gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
        
        # Salvando os dados em CSV no diretório de importação do Neo4j
        pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts1.csv', index=False, header=False)
        pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons1.csv', index=False, header=False)
        pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes1.csv', index=False, header=False)
        
        #Deletando o arquivo PDF após o processamento
        os.remove('tese 1.pdf')

        print(f"Loading triplas: {thesis_uri}")

        # Importanto os dados para o Neo4j
        #Excerpts
        query = """
        LOAD CSV FROM 'file:///excerpts1.csv'
        AS row
        MATCH  (t:Thesis{uri: row[0]})
        MERGE  (e:Exerpt{
                uri: row[1],
                text: row[2],
                page: row[3],
                lang: row[4]
                })-[r:BFO_0000050]-(t);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #Persons
        query = """
        LOAD CSV FROM 'file:///persons1.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Person{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #GPEs
        query = """
        LOAD CSV FROM 'file:///gpes1.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Place{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        # Removendo os arquivos CSV após a importação
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts1.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons1.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes1.csv')

        print(f"Total time: {pd.Timestamp.now() - start_time}")
        print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
        # Atualizando a hora parcial
        parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: a_percepcao_da_injuria_racial_e_racismo_entre_os_operadores_do_direito from https://www.teses.usp.br/teses/disponiveis/2/2140/tde-03102017-153049/publico/CamilaTavaresdeMouraBrasilMatos_PercepcaoInjuriaRacialeRacismo.pdf
Loading triplas: a_percepcao_da_injuria_racial_e_racismo_entre_os_operadores_do_direito
Total time: 0 days 00:03:45.045331
Partial time: 0 days 00:03:45.045331
1 - Downloading and extracting: nacionalismos_antiracistas_manoel_bomfim_e_manuel_gonzalez_prada_brasil_e_peru_na_passagem_para_o_seculo_xx from https://www.teses.usp.br/teses/disponiveis/8/8138/tde-12052023-101103/publico/2002_RicardoSequeiraBechelli.pdf
Loading triplas: nacionalismos_antiracistas_manoel_bomfim_e_manuel_gonzalez_prada_brasil_e_peru_na_passagem_para_o_seculo_xx
Total time: 0 days 00:17:16.214782
Partial time: 0 days 00:13:31.169451
2 - Downloading and extracting: valores_e_sentimentos_subjacentes_a_discriminacao_racial_um_estudo_na_perspectiva_dos_modelos_organiza

Transaction failed and will be retried in 0.8753953639925809s (ForsetiClient[transactionId=235270, clientId=3] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=235255, clientId=1]} on NODE_RELATIONSHIP_GROUP_DELETE(36844) because holders of that lock are waiting for ForsetiClient[transactionId=235270, clientId=3].
 Wait list:ExclusiveLock[
Client[235255] waits for [ForsetiClient[transactionId=235270, clientId=3]]])


Total time: 0 days 00:51:52.794218
Partial time: 0 days 00:05:54.549190
8 - Downloading and extracting: direitos_humanos_e_relacoes_raciais_uma_contribuicao_da_teoria_da_branquidade_para_a_analise_da_jurisprudencia_brasileira_sobre_a_conduta_da_discriminacao_racial_prevista_na_legislacao from https://www.teses.usp.br/teses/disponiveis/2/2140/tde-11042016-113657/publico/MARIA_LETICIA_PUGLISI_MUNHOZ.pdf
Loading triplas: direitos_humanos_e_relacoes_raciais_uma_contribuicao_da_teoria_da_branquidade_para_a_analise_da_jurisprudencia_brasileira_sobre_a_conduta_da_discriminacao_racial_prevista_na_legislacao
Total time: 0 days 01:09:14.582753
Partial time: 0 days 00:17:21.788535
9 - Downloading and extracting: escola_e_enfrentamento_do_racismo_as_experiencias_das_professoras_ganhadoras_do_premio_educar_para_a_igualdade_racial from https://www.teses.usp.br/teses/disponiveis/48/48134/tde-25032014-133053/publico/anexos.pdf
Loading triplas: escola_e_enfrentamento_do_racismo_as_experiencias_das_prof

DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: At C:\Users\facordei\.Neo4jDesktop\relate-data\dbmss\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\import\excerpts1.csv @ position 307084 -  there's a field starting with a quote and whereas it ends that quote there seems to be characters in that field after that ending quote. That isn't supported. This is what I read: '" '}